# Plotting the skeletons of certain neurons including selected neuropils
Here, neuron skeletons and neuropils are downloaded from the MANC connectome dataset. The neurons are plotted in three-dimensional plots which are saved to html files with plotly. 
The notebook requires the MANC dataset, which is imported as two hdf5 datasets. They can be created using the functions `fetch_all_connections()` and `fetch_all_neuron_props()` form my connectome_fun module. Further, a neuprint token is required. It should be stored in a text file at './accessory_files/neuprint_token.txt'. <br>
Note: This notebook was run with python 3.9.18 and navis 1.5.0. 

Autor: Bjarne Schultze <br>
Last modified: 29.11.2024

In [ ]:
import pandas as pd
import navis
import navis.interfaces.neuprint as neu
import plotly
from matplotlib.colors import to_rgba


# Save authentication token and create neuPrint+ client
with open("./accessory_files/neuprint_token.txt") as f:
    auth_token = f.readlines()[0]
# Create MANC neuprint client
neu_cl = neu.Client('neuprint.janelia.org', dataset='manc:v1.2.1', token=auth_token)

In [2]:
# Load the connections and neuron information
connections = pd.read_hdf('../additional_files/manc_1.2.1_connections.h5')
neuron_info = pd.read_hdf('../additional_files/manc_1.2.1_neurons.h5')

# Re-name AN08B031 to vPR13 (IDs checked to ensure same neurons in old and new MANC version)
connections.replace("AN08B031", "vPR13", inplace=True)
neuron_info.replace("AN08B031", "vPR13", inplace=True)

# Create ID and type dicts
neuron_dict = {}
# Iterate over the neuron types
for ntype in neuron_info['type'].unique():
    # Get the bodyIDs for all neurons of the same type
    bodyIds = neuron_info.loc[neuron_info['type']==ntype, 'bodyId'].to_numpy()
    # Assign the bodyIDs to the neuron type in the dict
    neuron_dict[ntype] = bodyIds

# Get IDs of vMS12 neurons
vms12_ids = neuron_dict["vMS12"]

In [ ]:
# Fetch ROIs
neuropils = [neu.fetch_roi(roi) for roi in ["WTct(UTct-T2)(L)", "WTct(UTct-T2)(R)"]] # , "IntTct", "LTct", "HTct(UTct-T3)(L)", "HTct(UTct-T3)(R)"]]
# Fetch neurons
vms12_meshes = neu.fetch_mesh_neuron(vms12_ids, lod=2, use_https=True)

# Store the body IDs 
vMS12_systype_in12a023 = [13518, 14183, 14534, 16920, 23069, 101397, 14126, 15364, 15875, 16317]
vMS12_systype_in12a014 = [12887, 12905, 13240, 13855]
vMS12_systype_in12a040 = [16889, 17931, 18452, 19042, 21441, 32714]
ipsilat = [13518, 14183, 14534, 16920, 23069, 101397]

# Sort the meshes in those of ipsilateral neurons and the others
ipsilat_meshes = []
other_meshes = []

for nmesh in vms12_meshes:
    # Modify name for the legend entry
    nmesh.name = f"{nmesh.name.split('_')[0]}"
    # Sort 
    if nmesh.id in ipsilat:
        ipsilat_meshes.append(nmesh)
    else:
        other_meshes.append(nmesh)

c:\Users\schul\anaconda3\envs\rms_analysis\lib\site-packages\python_jsonschema_objects\__init__.py:113: UserWarning: Schema id not specified. Defaulting to 'self'
  warnings.warn("Schema id not specified. Defaulting to 'self'")
c:\Users\schul\anaconda3\envs\rms_analysis\lib\site-packages\neuprint\client.py:609: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  neuron_props_val = df_results.iloc[0][0]
c:\Users\schul\anaconda3\envs\rms_analysis\lib\site-packages\neuprint\client.py:619: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  neuron_props_val = df_results.iloc[0][0]


Fetching:   0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
# Define colors for the ROIs (with transparency)
alpha = 0.2
roi_colors = [(0.1,0.9,0.8,alpha), (0.3,0.2,0.9,alpha)] # , (0.9,0.4,0.2,alpha), (0.9,0.8,0.2,alpha), (0.8,0.2,0.9,alpha), (0.3,0.7,0.1,alpha)]
# Define a set of colors
colorlist = ['firebrick','darkturquoise','limegreen','royalblue','hotpink','lightskyblue','gold','purple','slategray','tomato','yellowgreen','plum','gray',
                'deepskyblue','aqua','darkorchid','darkblue','olivedrab','wheat','lightcyan','seagreen','bisque']
mesh_colors = [ to_rgba(col) for col in colorlist[:len(vms12_meshes)] ]
mesh_colors.extend(roi_colors)


# Plot the neurons
fig3d = navis.plot3d([vms12_meshes, neuropils], color=mesh_colors, width=1000, height=700, inline=False)

# Save the plot as a html file 
_ = plotly.offline.plot(fig3d, filename='../figures_wip/vms12_neurons.html')

INFO  : Use the `.show()` method to plot the figure. (navis)


In [ ]:
# Define colors for the ROIs (with transparency)
alpha = 0.2
roi_colors = [(0.9,0.7,0.7,alpha), (0.7,0.7,0.9,alpha)] # , (0.9,0.4,0.2,alpha), (0.9,0.8,0.2,alpha), (0.8,0.2,0.9,alpha), (0.3,0.7,0.1,alpha)]
# Define a set of colors
colorlist = ['firebrick','darkturquoise','limegreen','royalblue','hotpink','darkorchid','gold','purple','slategray','tomato','yellowgreen','plum','gray',
                'deepskyblue','aqua','darkorchid','darkblue','olivedrab','wheat','lightcyan','seagreen','bisque']
mesh_colors = [ to_rgba(col) for col in colorlist[:len(ipsilat_meshes)] ]
mesh_colors.extend(roi_colors)


# Plot the neurons
fig3d = navis.plot3d([ipsilat_meshes, neuropils], color=mesh_colors, width=1200, height=800, inline=False)

fig3d.update_layout(
    scene = dict(
        xaxis = dict(visible=False),
        yaxis = dict(visible=False),
        zaxis =dict(visible=False)
        )
)

# Save the plot as a html file 
_ = plotly.offline.plot(fig3d, filename='../figures_wip/vms12_ipsilat.html')

INFO  : Use the `.show()` method to plot the figure. (navis)


In [50]:
# Define colors for the ROIs (with transparency)
alpha = 0.2
roi_colors = [(0.9,0.7,0.7,alpha), (0.7,0.7,0.9,alpha)] # , (0.9,0.4,0.2,alpha), (0.9,0.8,0.2,alpha), (0.8,0.2,0.9,alpha), (0.3,0.7,0.1,alpha)]
# Define a set of colors
colorlist = ['firebrick','darkturquoise','limegreen','royalblue','hotpink','darkorchid','gold','purple','slategray','tomato','yellowgreen','plum','gray',
                'deepskyblue','aqua','darkorchid','darkblue','olivedrab','wheat','lightcyan','seagreen','bisque']
mesh_colors = [ to_rgba(col) for col in colorlist[:len(other_meshes)] ]
mesh_colors.extend(roi_colors)


# Plot the neurons
fig3d = navis.plot3d([other_meshes, neuropils], color=mesh_colors, width=1200, height=800, inline=False)

fig3d.update_layout(
    scene = dict(
        xaxis = dict(visible=False),
        yaxis = dict(visible=False),
        zaxis =dict(visible=False)
        )
)

# Save the plot as a html file 
_ = plotly.offline.plot(fig3d, filename='../figures_wip/vms12_bilat.html')

INFO  : Use the `.show()` method to plot the figure. (navis)


In [4]:
# Fetch leg neuropils
legneuropils = [neu.fetch_roi(roi) for roi in ["LegNp(T1)(L)", "LegNp(T1)(R)", "LegNp(T2)(L)", "LegNp(T2)(R)", "LegNp(T3)(L)", "LegNp(T3)(R)"]]

In [11]:
# Create a list of colors
alpha = 0.2
mesh_colors2 = [ to_rgba("firebrick") for n in vms12_meshes ]
mesh_colors2.extend([ (0.9,0.9,0.9,alpha) for n in legneuropils ])
mesh_colors2.extend([(0.9,0.7,0.7,alpha), (0.7,0.7,0.9,alpha)])

# Plot the neurons
fig3d = navis.plot3d([vms12_meshes, legneuropils, neuropils], color=mesh_colors2, width=1200, height=800, inline=False)

fig3d.update_layout(
    scene = dict(
        xaxis = dict(visible=False),
        yaxis = dict(visible=False),
        zaxis = dict(visible=False)
        )
)

# Save the plot as a html file 
_ = plotly.offline.plot(fig3d, filename='../figures_wip/vms12_all.html')

INFO  : Use the `.show()` method to plot the figure. (navis)
